In [ ]:
import numpy as np
import random
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from qiskit.quantum_info import DensityMatrix, random_density_matrix
from qiskit.quantum_info.operators import Operator
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.optimize import minimize
from scipy.linalg import sqrtm

# Load the QST Dataset

In [2]:
QST_data = pd.read_csv("../data/qst_dataset.csv")
N = QST_data.shape[0]
QST_data.head()
# clip any small values to zero
eps = 1e-10

#create boolean mask
mask = QST_data.abs() < eps
QST_data[mask] =0

X_cols = []
y_cols = []

N_x = 36
N_y = 32
for i in range(N_x):
    X_cols.append(f"x{i}")
for i in range(N_y):
    y_cols.append(f"y{i}")

X = QST_data[X_cols]
y = QST_data[y_cols]

#split the data
X_train, X_test, y_train, y_test=  train_test_split(X, y, train_size=0.7, random_state=42)

# Reconstruct $\rho$ using Maximum Likelihood Estimation

## This approach parameterises the density matrix using a Cholesky Decomposition,

In [ ]:


#first convert into numpy arrays
X = X.to_numpy()
y = y.to_numpy()

#Restore the counts from the frequencies
shots = 1024
counts = (X * shots).astype(int)  # shape (N, 36)

# Build computational basis projectors for 2 qubits
proj = []
for m in range(4):
    P = np.zeros((4,4), dtype=complex)
    P[m, m] = 1
    proj.append(P)

# Define basis-change unitaries for X, Y, Z on one qubit
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
Sdg = np.array([[1, 0], [0, -1j]])
bases = {
    'X': H,
    'Y': Sdg @ H,
    'Z': np.eye(2)
}

# Build the POVM elements E_jm using the 9 joint Pauli unitaries
settings = []
for b1 in ['X','Y','Z']:
    for b2 in ['X','Y','Z']:
        U = np.kron(bases[b1], bases[b2])
        settings.append(U)

E = []
for U in settings:
    U_dag = U.conj().T
    for P in proj:
        E.append(U_dag @ P @ U)
# E contains the 36 POVM elements

# Map the cholesky parameterised vector to a PSD trace 1 density matrix
def params_to_rho(params):
    # params: length 16
    L = np.zeros((4,4), dtype=complex)
    idx = 0
    # diagonal entries (real, positive)
    for i in range(4):
        L[i, i] = params[idx]
        idx += 1
    # lower-triangular off-diagonals (real + imag)
    for i in range(1, 4):
        for j in range(i):
            re = params[idx]; im = params[idx+1]
            L[i, j] = re + 1j * im
            idx += 2
    rho = L @ L.conj().T
    return rho / np.trace(rho)

# COmpute NLL
def neg_log_likelihood(params, count):
    rho = params_to_rho(params)
    # avoid log(0) by clipping
    probs = np.array([np.real(np.trace(Ej @ rho)) for Ej in E])
    probs = np.clip(probs, 1e-12, 1.0)
    return -np.sum(count * np.log(probs))


N = X.shape[0]
#initialise the estimator
rho_est = np.zeros((N, 4, 4), dtype=complex)

for i in range(N):
    
    # initial guess: uniform identity
    init = np.zeros(16)
    init[:4] = np.sqrt(1/4)  # diagonal entries ~ sqrt(1/4)
    
    res = minimize(
        neg_log_likelihood, init, args=(counts[i],),
        method='L-BFGS-B',
        options={'maxiter': 500}
    )
    #reconstruct the density matrix from cholesky decomp
    rho_est[i] = params_to_rho(res.x)

# Recover target density matrices from y
rho_target = np.zeros((N, 4, 4), dtype=complex)
for i in range(N):
    real = y[i, :16].reshape(4,4)
    imag = y[i, 16:].reshape(4,4)
    rho_target[i] = real + 1j * imag

# compute Fidelity
def fidelity(rho1, rho2):
    sqrt_rho1 = sqrtm(rho1)
    F = np.trace(sqrtm(sqrt_rho1 @ rho2 @ sqrt_rho1))
    return np.real(F)**2

fidelities = np.array([fidelity(rho_est[i], rho_target[i]) for i in range(N)])

mean_fidelity = np.mean(fidelities)
std_fidelity  = np.std(fidelities)
print(f"Mean fidelity: {mean_fidelity:.4f} ± {std_fidelity:.4f}")


Mean fidelity: 0.8808 ± 0.0546
